## Carvana-MobileNet-UNet

In [1]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from optimizers.SGDAccumulate import SGDAccumulate
from models.Mobile_U_Net import mobile_u_net
from submit import generate_submit
from utilities import utils_masks
from utilities import losses

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


In [2]:
utils_masks.set_results_reproducible()

## Preparing Data

In [3]:
input_size = 128
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
df_train = pd.read_csv('inputs/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])#[:3000]
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

bboxes = None
bbox_file_path = 'inputs/train_bboxes.csv'
bboxes = utils_masks.get_bboxes(bbox_file_path)

def train_generator(batch_size):
    return utils_masks.train_generator(train_path, train_mask_path, ids_train_split, input_size, batch_size, bboxes)

def valid_generator(batch_size):
    return utils_masks.valid_generator(train_path, train_mask_path, ids_valid_split, input_size, batch_size, bboxes)

Training on 4070 samples
Validating on 1018 samples


## Create Model

In [4]:
model = mobile_u_net(input_size)

model.compile(optimizer=AdamAccumulate(accum_iters=4), 
              loss=losses.weighted_bce_dice_loss, 
              metrics=[losses.dice_value])

In [5]:
#model.summary()

## Fit Model

In [5]:
epochs = 5
batch_size = 16
run_name = utils_masks.get_run_name('weights/{}.hdf5', 'mobilenet-unet')
weights_path = 'weights/{}.hdf5'.format(run_name)

callbacks = [EarlyStopping(monitor='val_dice_value',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_value',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_value',
                             filepath=weights_path,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

#model.load_weights('weights/unet-2017-09-02-0322.hdf5')
#K.set_value(model.optimizer.lr, 0.01)

print('Starting run "{}"'.format(run_name))
model.fit_generator(generator=train_generator(batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Starting run "mobilenet-unet-2017-09-19-1322"


'model.fit_generator(generator=train_generator(batch_size),\n                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),\n                    epochs=epochs,\n                    verbose=1,\n                    callbacks=callbacks,\n                    validation_data=valid_generator(batch_size),\n                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))'

## Finetune weights of pretrained MobileNet.

In [6]:
model.load_weights('weights/mobilenet-unet-2017-09-19-0114.hdf5')

model.compile(optimizer=SGDAccumulate(lr=1e-4, momentum=0.9, accum_iters=4), 
              loss=losses.weighted_bce_dice_loss, 
              metrics=[losses.dice_value])

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 64, 32)    864         input_1[0][0]                    
____________________________________________________________________________________________________
conv1_bn (BatchNormalization)    (None, 64, 64, 32)    128         conv1[0][0]                      
____________________________________________________________________________________________________
conv1_relu (Activation)          (None, 64, 64, 32)    0           conv1_bn[0][0]                   
___________________________________________________________________________________________

In [7]:
layer_index = 58
for layer in model.layers[:layer_index]: layer.trainable=False
for layer in model.layers[layer_index:]: layer.trainable=True

In [28]:
model.layers[layer_index].name

'conv_dw_10'

In [ ]:
model.load_weights('weights/mobilenet-unet-2017-09-19-1216.hdf5')

In [ ]:
epochs = 100

print('Starting run "{}"'.format(run_name))
model.fit_generator(generator=train_generator(batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Starting run "mobilenet-unet-2017-09-19-1322"
Epoch 1/100
255/255 [==============================] - 160s - loss: 0.4740 - dice_value: 0.9936 - val_loss: 0.4653 - val_dice_value: 0.9951
Epoch 2/100
255/255 [==============================] - 152s - loss: 0.4743 - dice_value: 0.9936 - val_loss: 0.4653 - val_dice_value: 0.9951
Epoch 3/100
255/255 [==============================] - 152s - loss: 0.4748 - dice_value: 0.9936 - val_loss: 0.4653 - val_dice_value: 0.9951
Epoch 4/100
255/255 [==============================] - 152s - loss: 0.4738 - dice_value: 0.9936 - val_loss: 0.4653 - val_dice_value: 0.9951
Epoch 5/100
255/255 [==============================] - 152s - loss: 0.4749 - dice_value: 0.9936 - val_loss: 0.4653 - val_dice_value: 0.9951
Epoch 6/100
254/255 [============================>.] - ETA: 0s - loss: 0.4737 - dice_value: 0.9937
Epoch 00005: reducing learning rate to 9.99999974738e-06.
255/255 [==============================] - 151s - loss: 0.4738 - dice_value: 0.9937 - val_loss: 0